In [1]:
# AbacusResearch/RasGulla1-7b

In [1]:
# !pip install -qU transformers datasets loralib einops xformers

In [2]:
# !pip install git+https://github.com/huggingface/accelerate.git

In [3]:
# !pip install -qqq torch

In [4]:
# !pip install -i https://pypi.org/simple/ bitsandbytes

In [5]:
# !pip install peft

Installing torch and unsloth had been a nightmare since the torch version on colab breaks unsloth for some reason, following code taken from unsloth's huggingface account https://huggingface.co/datasets/unsloth/notebooks/blob/main/DPO_Zephyr_Unsloth_Example.ipynb

In [6]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

# DO NOT RUN, TO CHECK THE PREVIOUS ISSUE WITH TORCH

In [5]:
# this is old
import torch
print(torch.__version__)

2.1.0+cu121


# After unsloth's notebook torch, run from here

In [7]:
print(torch.__version__)

2.2.1+cu121


In [8]:
major_version, minor_version = torch.cuda.get_device_capability()
print(f"CUDA major version: {major_version}, minor version: {minor_version}")

CUDA major version: 7, minor version: 5


In [10]:
from unsloth import FastLanguageModel

# Using transformers, will switch to unsloth

In [20]:
# quantization
# quiantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [21]:
# tokenizer
# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

In [ ]:
# load model with 4bit quantization with the config
# model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", quantization_config=quiantization_config)

# unsloth

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-7b",
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)

Unsloth: You passed in `google/gemma-7b` and `load_in_4bit = True`.
We shall load `unsloth/gemma-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # rank stabilized
    loftq_config = None
)

Unsloth 2024.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [44]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="sha256_password.csv")

In [45]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'sha_256', 'password'],
        num_rows: 10000
    })
})

In [46]:
prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [59]:
def formatting_prompts_func(examples):
    instruction = "below is a sha256 representation of a password, please generate the password -"
    inputs       = examples["sha_256"]
    outputs      = examples["password"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        #print(text)
        texts.append(text)
    return { "text" : texts, }

In [60]:
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [64]:
dataset["train"][1]

{'Unnamed: 0': 1,
 'sha_256': '5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8',
 'password': 'password',
 'text': ' \n### Instruction:\nbelow is a sha256 representation of a password, please generate the password -\n\n### Input:\n5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8\n\n### Response:\npassword<eos>'}

In [27]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [49]:
dataset["train"]

Dataset({
    features: ['Unnamed: 0', 'sha_256', 'password', 'text'],
    num_rows: 10000
})

In [50]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 4096,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [51]:
trainer_stats = trainer.train()

Step,Training Loss
1,3.215800
2,3.121400
3,2.936900
4,2.623000
5,2.696800
6,2.538300
7,2.163300
8,2.057000
9,2.050500
10,2.041900


In [72]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "below is a sha256 representation of a password, please generate the password -",
        "4007d46292298e83da10d0763d95d5139fe0c157148d0587aa912170414ccba6", # input, AstonMartinDB11
        "", # output - expected response
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos> 
### Instruction:
below is a sha256 representation of a password, please generate the password -

### Input:
4007d46292298e83da10d0763d95d5139fe0c157148d0587aa912170414ccba6

### Response:
123456789<eos>
